<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/langchain/rag_with_milvus_and_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/langchain/full_text_search_with_langchain.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>
</a>


# Using full-text search with LangChain and Milvus

[Full-text search](https://milvus.io/docs/full-text-search.md#Full-Text-Search) retrieves documents with specific terms or phrases in text datasets and ranks results by relevance. It overcomes semantic search limitations to provide accurate, context-relevant results. Also, it simplifies vector searches, accepting raw text and automatically converting it into sparse embeddings without manual generation. By integrating full-text search with semantic-based dense vector search, you can enhance the accuracy and relevance of search results.

BM25 is an important ranking algorithm in full-text search. Using the BM25 algorithm for relevance scoring, this feature is particularly valuable in retrieval-augmented generation (RAG) scenarios, where it prioritizes documents that closely match specific search terms. 

Milvus 2.5 introduced the full-text search [feature](https://milvus.io/blog/introduce-milvus-2-5-full-text-search-powerful-metadata-filtering-and-more.md). As a further layer of framework, LangChain's Milvus integration has also launched this feature, making it easy to integrate full-text search into your application.

In this tutorial, we will show you how to use LangChain and Milvus to use full-text search into your application.

> - Full text search is available in Milvus Standalone and Milvus Distributed but not Milvus Lite, although adding it to Milvus Lite is on the roadmap.
> - Before reading this tutorial, you need to have a basic understanding of [full-text search](https://milvus.io/docs/full-text-search.md#Full-Text-Search). In addition, you also need to know the [basic usage](https://milvus.io/docs/basic_usage_langchain.md) of LangChain Milvus integration.



## Prerequisites

Before running this notebook, make sure you have the following dependencies installed:

In [1]:
! pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-text-splitters langchain-milvus langchain-openai langchain-voyageai bs4

> If you are using Google Colab, to enable dependencies just installed, you may need to **restart the runtime** (click on the "Runtime" menu at the top of the screen, and select "Restart session" from the dropdown menu).

We will use the models from OpenAI and VoyageAI. You should prepare the environment variables `OPENAI_API_KEY` from [OpenAI](https://platform.openai.com/docs/quickstart) and `VOYAGE_API_KEY` from [VoyageAI](https://docs.voyageai.com/docs/api-key-and-installation).

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-***********"
os.environ["VOYAGE_API_KEY"] = "pa-***********"

Install and start the Milvus server following this [guide](https://milvus.io/docs/install_standalone-docker-compose.md). And set your Milvus server `URI` (or optional `TOKEN`)

In [3]:
URI = "http://localhost:19530"
# TOKEN = ...

Prepare some examples documents:

In [4]:
from langchain_core.documents import Document

docs = [
    Document(page_content="I like apple", metadata={"foo": "bar"}),
    Document(page_content="I like banana", metadata={"foo": "baz"}),
    Document(page_content="I like orange", metadata={"foo": "qux"}),
]

## Initialization with BM25 Function
### Hybrid Search

Unlike simply passing an embedding to the `VectorStore`, the Milvus VectorStore provides a `builtin_function` parameter. Through this parameter, you can pass an instance of the BM25 function.

Here is a simple example of combining OpenAI embeddings with the BM25 function from Milvus:

In [5]:
from langchain_milvus import Milvus, BM25BuiltInFunction
from langchain_openai import OpenAIEmbeddings


vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
    builtin_function=BM25BuiltInFunction(),
    # `dense` is for OpenAI embeddings, `sparse` is the output field of BM25 function
    vector_field=["dense", "sparse"],
    connection_args={
        "uri": URI,
    },
    consistency_level="Strong",
    drop_old=True,
)

In the code above, we define an instance of `BM25BuiltInFunction` and pass it to the `Milvus` object. `BM25BuiltInFunction` is a lightweight wrapper class for the [`Function`](https://milvus.io/docs/manage-collections.md#Function) in Milvus.

You can specify the input and output fields for this function in the parameters of the `BM25BuiltInFunction` instance by passing the following two field parameters:
- `input_field_names` (str): The name of the input field, default is `text`. It indicates which field this function reads as input.
- `output_field_names` (str): The name of the output field, default is `sparse`. It indicates which field this function outputs the computed result to.

Note that in the Milvus initialization parameters mentioned above, we also specify `vector_field=["dense", "sparse"]`. Since the `sparse` field is the output field defined by the `BM25BuiltInFunction`, the other `dense` field will be automatically assigned to the output field of OpenAIEmbeddings.

In practice, especially when combining multiple embeddings or functions, we recommend clearly specifying the input and output fields for each function to avoid confusion.

In the following example, it specifies the input and output fields of BM25BuiltInFunction, and three vector fields, which makes it clear which field each built-in function and each vector embedding.


In [6]:
from langchain_voyageai import VoyageAIEmbeddings

embedding1 = OpenAIEmbeddings(model="text-embedding-3-large")
embedding2 = VoyageAIEmbeddings(model="voyage-3")

vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=[embedding1, embedding2],
    builtin_function=BM25BuiltInFunction(
        input_field_names="text", output_field_names="sparse"
    ),
    text_field="text",  # `text` is the input field name of BM25BuiltInFunction
    # `sparse` is the output field name of BM25BuiltInFunction, and `dense1` and `dense2` are the output field names of OpenAIEmbeddings and VoyageAIEmbeddings
    vector_field=["dense1", "dense2", "sparse"],
    connection_args={
        "uri": URI,
    },
    consistency_level="Strong",
    drop_old=True,
)

vectorstore.vector_fields

['dense1', 'dense2', 'sparse']

In this example, we have three vector fields. Among them, `sparse` is used as the output field for `BM25BuiltInFunction`, while the other two, `dense1` and `dense2`, are automatically assigned as the output fields for `OpenAIEmbeddings` and `VoyageAIEmbeddings`, respectively.  

In this way, you can define multiple vector fields and assign different combinations of embeddings or functions to them, enabling hybrid search.


When performing hybrid search, we just pass in the query text and optionally set the topK and reranker parameters. The `vectorstore` instance will automatically handle the vector embeddings and built-in functions and finally use a reranker to refine the results. From the user's end, we don't need to care about the underlying implementation details of the searching process.

In [7]:
vectorstore.similarity_search(
    "Do I like oranges?", k=1
)  # , ranker_type="weighted", ranker_params={"weights":[0.3, 0.3, 0.4]})

[Document(metadata={'foo': 'qux', 'pk': 454646931479251686}, page_content='I like orange')]

For more information about how to use the hybrid search, you can refer to the [Hybrid Search introduction](https://milvus.io/docs/multi-vector-search.md#Hybrid-Search) and this [LangChain Milvus hybrid search tutorial](https://milvus.io/docs/milvus_hybrid_search_retriever.md) .

### BM25 search without embedding

If you want to perform lexical frequency-based full-text search using only a single BM25 function without using any embedding-based semantic similarity search, you can set the embedding parameter input to `None` and keep only the `builtin_function` parameter input as the BM25 function instance. For example:  

In [8]:
vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=None,
    builtin_function=BM25BuiltInFunction(
        output_field_names="sparse",
    ),
    vector_field="sparse",
    connection_args={
        "uri": URI,
    },
    consistency_level="Strong",
    drop_old=True,
)

vectorstore.vector_fields

['sparse']

## Customize analyzer

Analyzers are essential tools in text processing that convert raw text into structured, searchable formats. They play a key role in enabling efficient indexing and retrieval by breaking down input text into tokens and refining these tokens through a combination of tokenizers and filters. For more information, you can refer [this guide](https://milvus.io/docs/analyzer-overview.md#Analyzer-Overview) to learn more about analyzers in Milvus.

Milvus supports two types of analyzers: **Built-in Analyzers**  and **Custom Analyzers**. By default, the `BM25BuiltInFunction` will use the [default standard analyzer](https://milvus.io/docs/standard-analyzer.md), which makes it effective for most languages. 

However, if you want to use a different analyzer or customize the analyzer, you can pass in the `analyzer_params` parameter in the `BM25BuiltInFunction` initialization.



In [9]:
analyzer_params_custom = {
    "tokenizer": "standard",
    "filter": [
        "lowercase",  # Built-in filter
        {"type": "length", "max": 40},  # Custom filter
        {"type": "stop", "stop_words": ["of", "to"]},  # Custom filter
    ],
}


vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
    builtin_function=BM25BuiltInFunction(
        output_field_names="sparse",
        enable_match=True,
        analyzer_params=analyzer_params_custom,
    ),
    vector_field=["dense", "sparse"],
    connection_args={
        "uri": URI,
    },
    consistency_level="Strong",
    drop_old=True,
)

We can take a look at the schema of the Milvus collection and make sure the customized analyzer is set up correctly.

In [10]:
vectorstore.col.schema

{'auto_id': True, 'description': '', 'fields': [{'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535, 'enable_match': True, 'enable_analyzer': True, 'analyzer_params': {'tokenizer': 'standard', 'filter': ['lowercase', {'type': 'length', 'max': 40}, {'type': 'stop', 'stop_words': ['of', 'to']}]}}}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'dense', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}, {'name': 'sparse', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>, 'is_function_output': True}, {'name': 'foo', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}], 'enable_dynamic_field': False, 'functions': [{'name': 'bm25_function_7c99f463', 'description': '', 'type': <FunctionType.BM25: 1>, 'input_field_names': ['text'], 'output_field_names': ['sparse'], 'params': {}}]}

For more concept details, e.g., `analyzer`, `tokenizer`, `filter`, `enable_match`, `analyzer_params`, please refer to the [analyzer documentation](https://milvus.io/docs/analyzer-overview.md).

## Best practice of RAG
We have learned how to use the basic BM25 build-in function in LangChain and Milvus. Let's introduce the best practice of RAG in combination with this usage.


![](../../../../images/advanced_rag/hybrid_and_rerank.png)

This diagram shows the Hybrid Retrieve & Reranking process, combining BM25 for keyword matching and vector search for semantic retrieval. Results from both methods are merged, reranked, and passed to an LLM to generate the final answer.

Hybrid search balances precision and semantic understanding, improving accuracy and robustness for diverse queries. It retrieves candidates with BM25 full-text search and vector search, ensuring both semantic, context-aware, and accurate retrieval.

Let's get started with an example.


### Prepare the data

We use the Langchain WebBaseLoader to load documents from web sources and split them into chunks using the RecursiveCharacterTextSplitter.


In [11]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()
# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

# Let's take a look at the first document
docs[1]

USER_AGENT environment variable not set, consider setting it to identify your requests.


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search)

### Load the document into Milvus vector store
As the introduction above, we initialize and load the prepared documents into Milvus vector store, which contains two vector fields: `dense` is for the OpenAI embedding and `sparse` is for the BM25 function.

In [12]:
vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
    builtin_function=BM25BuiltInFunction(),
    vector_field=["dense", "sparse"],
    connection_args={
        "uri": URI,
    },
    consistency_level="Strong",
    drop_old=True,
)

### Build RAG chain
We prepare the LLM instance and prompt, then conbine them into a RAG pipeline using the LangChain Expression Language.

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Initialize the OpenAI language model for response generation
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Define the prompt template for generating AI responses
PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

# Create a PromptTemplate instance with the defined template and input variables
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
# Convert the vector store to a retriever
retriever = vectorstore.as_retriever()


# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

Use the LCEL(LangChain Expression Language) to build a RAG chain.

In [14]:
# Define the RAG (Retrieval-Augmented Generation) chain for AI response generation
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

Invoke the RAG chain with a specific question and retrieve the response

In [16]:
query = "What is PAL and PoT?"
res = rag_chain.invoke(query)
res

'PAL (Program-aided Language models) and PoT (Program of Thoughts prompting) are approaches that involve using language models to generate programming language statements to solve natural language reasoning problems. This method offloads the solution step to a runtime, such as a Python interpreter, effectively decoupling complex computation and reasoning. PAL and PoT rely on language models with strong coding skills to perform these tasks.'

Congratulations! You have built a hybrid(dense vector + sparse bm25 function) search RAG chain powered by Milvus and LangChain.